In [4]:
from importlib import reload
import medvqa

In [10]:
reload(medvqa.datasets.mimiccxr.report_utils)

<module 'medvqa.datasets.mimiccxr.report_utils' from '/home/pamessina/medvqa/medvqa/datasets/mimiccxr/report_utils.py'>

In [11]:
from medvqa.datasets.mimiccxr.report_utils import ReportFactsDisplayer

In [12]:
rfd = ReportFactsDisplayer(
    preprocessed_reports_filepath='/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json',
    extracted_facts_filepaths=[
        '/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4-0613_parsed_sentences__v2.jsonl',
        '/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_parsed_sentences__v2.jsonl',
        '/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_parsed_sentences__v2(uniform).jsonl',
        '/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/extracted_facts_Seq2Seq(t5-small)_256_1_20230704_081617.jsonl',
    ],
)

Loading preprocessed reports from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/background_findings_and_impression_20230612_174143.json...
Loading extracted facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4-0613_parsed_sentences__v2.jsonl...
Loading extracted facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_parsed_sentences__v2.jsonl...
Loading extracted facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-3.5-turbo-0613_parsed_sentences__v2(uniform).jsonl...
Loading extracted facts from /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/extracted_facts_Seq2Seq(t5-small)_256_1_20230704_081617.jsonl...
Building facts...
Building facts to report ids...


227835it [00:54, 4148.79it/s]

Loaded 227835 preprocessed reports
Loaded 677694 sentences with extracted facts
Loaded 578802 unique facts
Loaded 578802 facts with report ids


In [16]:
# rfd.display(85687)
rfd.display()

RID: 195097

{'background': 'history: -year-old man with MRSA bacteremia.',
 'findings': 'AP chest compared to :. Relatively global pulmonary '
             'opacification progressing since is most readily explained by '
             'pulmonary edema. Concurrent pneumonia might not be recognized. '
             'Moderate bilateral pleural effusion, left greater than right has '
             'increased minimally since , worsened since . ET tube is in '
             'standard placement. Nasogastric tube passes into the stomach and '
             'out of view. No pneumothorax. Moderate cardiomegaly, unchanged.',
 'impression': '',
 'path': '/mnt/data/mimic-cxr/files/p18/p18769510/s51811670.txt'}

                                 FINAL REPORT
 AP CHEST 1:27 A.M. ON ___
 
 HISTORY:  ___-year-old man with MRSA bacteremia.
 
 IMPRESSION:
 AP chest compared to ___:
 
 Relatively global pulmonary opacification progressing since ___ is
 most readily explained by pulmonary edema.  Concurrent pneu

In [17]:
from medvqa.utils.files import load_jsonl

In [39]:
tmp = load_jsonl("/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578733,58628071).improved_comparison(6526297).jsonl")

In [40]:
len(tmp)

578733

In [25]:
from pprint import pprint

In [34]:
aux = []
for x in tmp:
    if x['metadata']['prev_study_comparison?'] == 'yes':\
        aux.append(x)

In [36]:
import random
random.choice(aux)

{'fact': 'stable right-sided pleural thickening',
 'metadata': {'anatomical location': 'right',
  'detailed observation': 'stable right-sided pleural thickening',
  'short observation': 'pleural thickening',
  'category': 'anatomical finding',
  'health status': 'abnormal',
  'prev_study_comparison?': 'yes',
  'comparison status': 'stable'},
 'extraction_method': 't5-small-finetuned'}

In [38]:
aux[4000]

{'fact': 'no longer net volume loss in the left hemithorax',
 'metadata': {'anatomical location': 'left hemithorax',
  'detailed observation': 'no longer net volume loss',
  'short observation': 'no longer net volume loss',
  'category': 'anatomical finding',
  'health status': 'normal',
  'prev_study_comparison?': 'yes',
  'comparison status': 'resolved'},
 'extraction_method': 'gpt-3.5-turbo-0613'}

In [51]:
tmp[-1]

{'fact': 'long-standing calcified granuloma in the right lower lobe',
 'metadata': {'anatomical location': 'right lower lobe',
  'detailed observation': 'long-standing calcified granuloma',
  'short observation': 'calcified granuloma',
  'category': 'anatomical finding',
  'health status': 'unknown',
  'prev_study_comparison?': 'no',
  'comparison status': ''},
 'extraction_method': 't5-small-finetuned',
 'improved_comparison': {'comparison': 'no comparison',
  'extraction_method': 't5-small-finetuned'}}

In [42]:
from collections import Counter

In [47]:
len(set(x['metadata']['anatomical location'] for x in tmp))

28348

In [45]:
len(set(x['metadata']['detailed observation'] for x in tmp))

428912

In [46]:
len(set(x['metadata']['short observation'] for x in tmp))

214604

In [52]:
aux2 = []
for x in tmp:
    try:
        y = x['improved_comparison']['comparison']
    except KeyError:
        y = x['metadata']['comparison status']
    if y == '':
        y = 'no comparison'
    aux2.append(y)

In [58]:
for x,y in sorted(list(Counter(aux2).items()), key=lambda x:x[1], reverse=True):
    print(f'{y}\t{x}')

332669	no comparison
62095	stable/unchanged
28092	worsened
25292	resolved
23833	improved
23214	position changed
19614	unclear comparison
17690	new finding
14540	increase
9434	decrease
7434	progressed
5657	larger
5475	smaller
3548	reappeared
146	other


In [60]:
aux3 = [x['metadata']['category'] for x in tmp]

In [61]:
for x,y in sorted(list(Counter(aux3).items()), key=lambda x:x[1], reverse=True):
    print(f'{y}\t{x}')

379530	anatomical finding
81103	tubes and lines
48601	technical assessment
34586	disease
34035	device
413	
193	symptom
159	procedure
85	unknown
14	health status
5	foreign body
3	surgical procedure
2	complication
2	surgery
1	treatment
1	normal


In [62]:
aux4 = [x['metadata']['health status'] for x in tmp]

In [63]:
for x,y in sorted(list(Counter(aux4).items()), key=lambda x:x[1], reverse=True):
    print(f'{y}\t{x}')

303619	abnormal
163015	unknown
75101	normal
36706	ambiguous
263	
24	improved
5	resolved


In [64]:
tmp2 = load_jsonl("/home/pamessina/medvqa-workspace/cache/mimiccxr/openai/gpt-4-0613_paraphrased_observations__two-or-more-words_cluster-balanced.jsonl")

In [73]:
x = 9888
print(tmp2[x]['metadata']['query'])
print('----')
for y in tmp2[x]['parsed_response']:
    print(y)

abnormality potentially explained by an inflammatory process such as atypical infection
----
The irregularity might be due to an inflammatory condition like an unusual infection
Possible inflammation such as an atypical infection could account for the abnormality
The abnormality could be a result of an inflammatory process, perhaps an unusual infection
An unusual infection or other inflammatory process might explain the abnormality
The abnormality might be attributable to an inflammation, such as an atypical infection
The observed abnormality could be caused by an inflammatory process like an unusual infection
An inflammatory process like an atypical infection might be the cause of the observed abnormality
The abnormality could potentially be due to an inflammatory condition like an atypical infection
The abnormality could be related to an inflammatory process, possibly an unusual infection
An atypical infection or similar inflammatory process could explain the abnormality
The abnormal